In [1]:
!pip install -q loralib
!pip install -q sentencepiece
!pip install -q bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done




---



In [2]:
import transformers

In [3]:
assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"

In [ ]:
from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("jiuhai/florence-vl-3b-pretrain")
model = LlamaForCausalLM.from_pretrained(
    "jiuhai/florence-vl-3b-pretrain",
    load_in_4bit=True,
    device_map="auto",
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
You are using a model of type llava_phi to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.
Missing required keys in `rope_scaling`: 'factor'


In [27]:
model.config.to_json_file("adapter_config.json")

In [34]:
#model = PeftModel.from_pretrained(model, "jiuhai/florence-vl-3b-pretrain")

In [ ]:

from transformers import AutoProcessor, AutoModelForCausalLM
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

In [41]:
from PIL import Image
import requests
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

In [44]:
task_prompt = '<CAPTION>'
run_example(task_prompt)

NameError: name 'processor' is not defined

In [35]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{instruction}

### Entrada:
{input}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{instruction}

### Resposta:"""

In [36]:
from pprint import pprint

In [37]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        pprint("Resposta: " + output.split("### Resposta:")[1].strip())

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [38]:
evaluate(input("Instrução: "))

Instrução: hello


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.75` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.flo

('Resposta: '
 'congcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongcongbpcongbpcongcongbpcongbpcongcongbpcongbpcongbpcongbpcongbpcongbpcongbpcongbpcongbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpbpikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikaiikai')
